In [1]:


date = "5_13_25"
feature_type = "alpha_power"


model_patch_sizes = ["10ms", "100ms", "1sec"]
files = [f"{x}_{feature_type}_probes_for_all_checkpoints_{date}.pkl" for x in model_patch_sizes]

In [ ]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style("whitegrid")

plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['font.size'] = 14

fig, ax = plt.subplots()

_df_rows = []

for f, patch_size in zip(files, model_patch_sizes):
    all_probes = pickle.load(open(f, 'rb'))    
    for key, item in all_probes.items():
        # # print(key, item.keys())
        all_mse = item["model_metrics_across_folds"]["sequence_level"]["mean_squared_errors"]
        all_mae = item["model_metrics_across_folds"]["sequence_level"]["mean_absolute_errors"]
        all_r2 = item["model_metrics_across_folds"]["sequence_level"]["r2_scores"]
        for (mse, mae, r2) in zip(all_mse, all_mae, all_r2):
            tmp = {}
            tmp['x'] = key
            tmp['mse'] = mse
            tmp['mae'] = mae
            tmp['r2'] = r2
            tmp['patch_size'] = patch_size
            _df_rows.append(tmp)
            
plot_df = pd.DataFrame(_df_rows)

sns.lineplot(
    data=plot_df, x='x', y='mse', errorbar='sd', err_style="band",
    hue="patch_size",
    marker="patch_size", 
    # label=f"Patch size: {model_patch_size}",
    ax=ax,
)

plt.xticks(rotation=20)
plt.xlabel("Checkpoints", fontsize=20)
plt.ylabel(f"{feature_type} Probe MSE (heldout set)", fontsize=20)
plt.show()